In [25]:
import pandas as pd
import undetected_chromedriver as uc
from bs4 import BeautifulSoup 
from time import sleep
from datetime import date

In [26]:
# Uses undetected_chromedriver to bypass cloudflare and get source from dynamic content
options = uc.ChromeOptions() 
options.headless = True 
driver = uc.Chrome(use_subprocess=True, options=options) 

C:\Users\Vinh\AppData\Local\Temp\ipykernel_10696\3125079143.py:3: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


In [27]:
# Dataframe
columns = ['Course Title', 'Course Headline',  'Rating', 'Reviews', 'Course Length', 'Lectures', 'Level', 'Instructor' ]
df=pd.DataFrame(columns=columns)

In [28]:
# Data extractor
def scrapeCourses(courses):
    rows = []
    for course in courses:
        row = []

        try:
            course_info = course.find('div',class_="course-card--main-content--2XqiY")
        except:
            course_info = None
            continue

        try:         
            course_title=course_info.find("div", class_="ud-sr-only").previousSibling.text
        except: course_title=None

        try:
            course_head=course_info.find("span",{'data-purpose':'seo-headline'}).text
        except:
            course_head = None

        try:
            course_rating=course_info.find("span",{'data-purpose':'seo-rating'}).text
        except:
            course_rating = None

        try:
            course_reviews=course_info.find("span",{'data-purpose':'seo-num-reviews'}).text
        except:
            course_reviews = None

        try:
            course_hours=course_info.find("span",{'data-purpose':'seo-content-info'}).text
        except:
            course_hours=None

        try: 
            course_lectures = course_info.find("span",{'data-purpose':'seo-num-lectures'}).text
        except: 
            course_lectures= None
        try:
            course_level = course_info.find("span",{'data-purpose':'seo-instructional-level'}).text
        except: course_level= None

        try:
            course_instructor=course_info.find("div",{'data-purpose':"safely-set-inner-html:course-card:visible-instructors"}).text
        except:
            course_instructor= None

        row = [course_title, course_head, course_rating, course_reviews, course_hours, course_lectures, course_level, course_instructor]
        rows.append(row)
        
    return rows
    

In [29]:
# Params
queries = ['programming',
'python',
'javascript',
'web development',
'data structures',
'algoritihms',
'java',
'ruby',
'html',
'css',
'ruby on rails',
'react',
'SQL',
'mongoDB',
'node',
'graphQL',
'flask',
'django',
'full stack web developer',
'C#'
]
queries = [query.replace(' ', '+') for query in queries ]
page_limit= 10

queries

['programming',
 'python',
 'javascript',
 'web+development',
 'data+structures',
 'algoritihms',
 'java',
 'ruby',
 'html',
 'css',
 'ruby+on+rails',
 'react',
 'SQL',
 'mongoDB',
 'node',
 'graphQL',
 'flask',
 'django',
 'full+stack+web+developer',
 'C#']

In [30]:
# Driver
for query in queries:
    for page in range(1,page_limit+1):
        driver.get(f'https://www.udemy.com/courses/search/?p={page}&q={query}&src=ukw')
        #driver.get("https://www.udemy.com/topic/javascript/")
        sleep(5)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        courses_list = soup.find('div', {'class': 'course-list--container--3zXPS'})
        courses = courses_list.find_all('div', {'class': 'course-card--container--1QM2W'})
        rows = scrapeCourses(courses)
        tempdf = pd.DataFrame(rows,columns=columns)
        df=pd.concat([df,tempdf], ignore_index=True)

driver.close()

In [31]:
df

,Course Title,Course Headline,Rating,Reviews,Course Length,Lectures,Level,Instructor
0,Pre-Programming: Everything you need to know b...,Increase your chance of success learning to co...,Rating: 4.6 out of 5,10634 reviews,6.5 total hours,87 lectures,Beginner,Evan Kimbrell
1,Learn Python Programming Masterclass,This Python For Beginners Course Teaches You T...,Rating: 4.6 out of 5,96573 reviews,73 total hours,542 lectures,All Levels,"Tim Buchalka, Jean-Paul Roberts, Tim Buchalka'..."
2,Beginning C++ Programming - From Beginner to B...,Obtain Modern C++ Object-Oriented <strong>Prog...,Rating: 4.6 out of 5,56157 reviews,46 total hours,305 lectures,All Levels,"Tim Buchalka's Learn Programming Academy, Dr. ..."
3,Java Programming Masterclass updated to Java 17,Learn Java In This Course And Become a Compute...,Rating: 4.5 out of 5,176243 reviews,101 total hours,529 lectures,All Levels,"Tim Buchalka, Tim Buchalka's Learn Programming..."
4,Automate the Boring Stuff with Python Programming,A practical <strong>programming</strong> cours...,Rating: 4.7 out of 5,105085 reviews,9.5 total hours,51 lectures,All Levels,Al Sweigart
...,...,...,...,...,...,...,...,...
5355,The Magical World of Pointers in C,Understanding and Mastering Pointers in <stron...,Rating: 4.2 out of 5,9 reviews,1 total hour,25 lectures,All Levels,Vishal Asrani
5356,C Programming - skills test (beginner/intermed...,Test your skills and become an expert in <stro...,Rating: 4.0 out of 5,1 review,66 questions,None,Beginner,Bert Broer
5357,Sorting Algorithms using Java & C: Make Your B...,Learn Analysis of Algorithm using asymptotic n...,Rating: 4.6 out of 5,189 reviews,6.5 total hours,59 lectures,Intermediate,Shibaji Paul
5358,Arduino Embedded C With Electronics/Schematics...,"Learn all about schematics, Printed Circuit Bo...",Rating: 4.7 out of 5,336 reviews,4 total hours,22 lectures,Beginner,Daniel McCarthy


In [32]:
# Output
df.to_csv(f'output/udemy_courses_{queries[0]}_{queries[1]}_{date.today()}.csv')
df.to_json(f'output/udemy_courses_{queries[0]}_{queries[1]}_{date.today()}.json')